In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from sklearn.model_selection import train_test_split

2022-11-13 16:42:16.734525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from IPython.display import HTML

In [4]:
model = tf.keras.Sequential([
            tfl.Flatten(input_shape=(23, 7)),
            tfl.Dense(100, activation='relu'),
            tfl.Dense(80, activation='relu'), 
            tfl.Dense(2,activation=None)
        ])

2022-11-13 16:42:32.976317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model_string = f"models/basic_model/epochs10.ckpt"

model.load_weights(model_string)

In [6]:
feature_arr = np.load("./cleaned_data/feature_arr_ids.npy")
target_arr = np.load("./cleaned_data/target_arr_ids.npy")

np.nan_to_num(feature_arr, copy=False, nan=0)
np.nan_to_num(target_arr, copy=False, nan=0)

x_preprocess_train, x_preprocess_test, y_train, y_test = train_test_split(feature_arr, target_arr, test_size=0.3, random_state=5)
x_preprocess_test, x_preprocess_val, y_test, y_val = train_test_split(x_preprocess_test, y_test, test_size=0.5, random_state=5)

In [7]:
''' 
Feature list:
['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir']
'''

x_train = x_preprocess_train[:,:,4:11]
x_val = x_preprocess_val[:,:,4:11]
x_test = x_preprocess_test[:,:,4:11]

train_mu = np.mean(x_train.reshape(-1,x_train.shape[-1]),axis=0)
train_std = np.std(x_train.reshape(-1,x_train.shape[-1]),axis=0)

x_train[:,:,1:] = (x_train[:,:,1:] - train_mu[1:])/train_std[1:]
x_val[:,:,1:] = (x_val[:,:,1:] - train_mu[1:])/train_std[1:]
x_test[:,:,1:] = (x_test[:,:,1:] - train_mu[1:])/train_std[1:]

In [8]:
def create_ngs_df():

    ngs_df = pd.read_csv("data/week1.csv", engine='c')
    print(f"dataframe shape = {ngs_df.shape}")

    for i in range(2,9):
        ngs_df = pd.concat([ngs_df,pd.read_csv(f"data/week{i}.csv", engine='c') ])
    return ngs_df

ngs_df = create_ngs_df()

dataframe shape = (1118122, 16)


In [23]:
val_df = pd.DataFrame(x_preprocess_val.reshape(x_val.shape[0], -1)).sort_values([0,1,2])

In [10]:
preds = model.predict(x_val)

1317/1317 [==============================] - 4s 2ms/step


In [13]:
preds[2]

array([-10.983147 ,  -1.0058748], dtype=float32)

In [9]:
sample = x_preprocess_val[0]

game_id = sample[0][0]
play_id = sample[0][1]
frame_id = sample[0][2]

In [9]:
play_df = ngs_df.query("gameId == @game_id and playId == @play_id")

In [13]:
play_df.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
711505,2021102408,3720,37101.0,1,2021-10-24 23:15:56,2.0,HOU,right,51.98,5.97,0.0,0.0,0.0,97.69,96.44,None
711506,2021102408,3720,37101.0,2,2021-10-24 23:15:57,2.0,HOU,right,51.98,5.97,0.0,0.0,0.0,97.69,98.01,None
711507,2021102408,3720,37101.0,3,2021-10-24 23:15:57,2.0,HOU,right,51.98,5.97,0.0,0.0,0.0,98.42,95.12,None
711508,2021102408,3720,37101.0,4,2021-10-24 23:15:57,2.0,HOU,right,51.98,5.97,0.0,0.0,0.0,98.42,93.05,None
711509,2021102408,3720,37101.0,5,2021-10-24 23:15:57,2.0,HOU,right,51.98,5.97,0.0,0.0,0.0,98.42,93.09,None


In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
